In [1]:
from __future__ import print_function
from six.moves import range
from PIL import Image

import torch.backends.cudnn as cudnn
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import os
import time

import sys
sys.path.append('/home/michaelcai/GANtor-Arts-Center/src/code/')

import numpy as np
import torchfile

from miscc.config import cfg, cfg_from_file
from miscc.utils import mkdir_p
from miscc.utils import weights_init
from miscc.utils import save_img_results, save_model
from miscc.utils import KL_loss
from miscc.utils import compute_discriminator_loss, compute_generator_loss

from tensorboard import summary
from tensorboardX import FileWriter

import torchvision.utils as vutils

In [2]:
import pandas as pd
import csv

In [3]:
csv_path = '/home/michaelcai/GANtor-Arts-Center/data/genre_train.csv'
filenames, classes = [], []
with open(csv_path) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        filenames.append(str(row[0])) 
        classes.append(int(row[1]))


In [4]:
folders = {0: 'abstract_painting', 
           1: 'cityscape', 
           2: 'genre_painting', 
           3: 'illustration', 
           4: 'landscape', 
           5: 'nude_painting', 
           6: 'portrait',
           7: 'religious_painting',
           8: 'sketch_and_study',
           9: 'still_life'}
print(folders)

{0: 'abstract_painting', 1: 'cityscape', 2: 'genre_painting', 3: 'illustration', 4: 'landscape', 5: 'nude_painting', 6: 'portrait', 7: 'religious_painting', 8: 'sketch_and_study', 9: 'still_life'}


In [5]:
classnums = [0] * 10

In [10]:
import subprocess
for i in range(len(filenames)):
    img_path = '/home/michaelcai/GANtor-Arts-Center/data/wikiart/' + filenames[i]
    img_name = filenames[i][filenames[i].rfind('/'):]
    save_path = '/home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/' + folders[classes[i]] + str(i)
    subprocess.call(['cp', img_path, save_path])
    if i % 100 == 0:
        print("finished iteration " + str(i) + "/" + str(len(filenames)))
    classnums[classes[i]] += 1

finished iteration 0/64995
finished iteration 100/64995
finished iteration 200/64995
finished iteration 300/64995
finished iteration 400/64995
finished iteration 500/64995
finished iteration 600/64995
finished iteration 700/64995
finished iteration 800/64995
finished iteration 900/64995
finished iteration 1000/64995
finished iteration 1100/64995
finished iteration 1200/64995
finished iteration 1300/64995
finished iteration 1400/64995
finished iteration 1500/64995
finished iteration 1600/64995
finished iteration 1700/64995
finished iteration 1800/64995
finished iteration 1900/64995
finished iteration 2000/64995
finished iteration 2100/64995
finished iteration 2200/64995
finished iteration 2300/64995
finished iteration 2400/64995
finished iteration 2500/64995
finished iteration 2600/64995
finished iteration 2700/64995
finished iteration 2800/64995
finished iteration 2900/64995
finished iteration 3000/64995
finished iteration 3100/64995
finished iteration 3200/64995
finished iteration 330

finished iteration 26800/64995
finished iteration 26900/64995
finished iteration 27000/64995
finished iteration 27100/64995
finished iteration 27200/64995
finished iteration 27300/64995
finished iteration 27400/64995
finished iteration 27500/64995
finished iteration 27600/64995
finished iteration 27700/64995
finished iteration 27800/64995
finished iteration 27900/64995
finished iteration 28000/64995
finished iteration 28100/64995
finished iteration 28200/64995
finished iteration 28300/64995
finished iteration 28400/64995
finished iteration 28500/64995
finished iteration 28600/64995
finished iteration 28700/64995
finished iteration 28800/64995
finished iteration 28900/64995
finished iteration 29000/64995
finished iteration 29100/64995
finished iteration 29200/64995
finished iteration 29300/64995
finished iteration 29400/64995
finished iteration 29500/64995
finished iteration 29600/64995
finished iteration 29700/64995
finished iteration 29800/64995
finished iteration 29900/64995
finished

finished iteration 53300/64995
finished iteration 53400/64995
finished iteration 53500/64995
finished iteration 53600/64995
finished iteration 53700/64995
finished iteration 53800/64995
finished iteration 53900/64995
finished iteration 54000/64995
finished iteration 54100/64995
finished iteration 54200/64995
finished iteration 54300/64995
finished iteration 54400/64995
finished iteration 54500/64995
finished iteration 54600/64995
finished iteration 54700/64995
finished iteration 54800/64995
finished iteration 54900/64995
finished iteration 55000/64995
finished iteration 55100/64995
finished iteration 55200/64995
finished iteration 55300/64995
finished iteration 55400/64995
finished iteration 55500/64995
finished iteration 55600/64995
finished iteration 55700/64995
finished iteration 55800/64995
finished iteration 55900/64995
finished iteration 56000/64995
finished iteration 56100/64995
finished iteration 56200/64995
finished iteration 56300/64995
finished iteration 56400/64995
finished

In [11]:
print(classnums)

[5153, 4764, 11217, 1958, 13780, 2001, 14675, 6783, 4092, 2900]


In [12]:
sum = 0
for cls in classnums:
    sum += cls
print(sum)

67323


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

